# Run CoCiP over a flight

This tutorial walks through an example of running the [Contrail Cirrus Predicition (CoCiP)](https://gmd.copernicus.org/articles/5/543/2012/) model evaluation along a flight trajectory.

## References

- Schumann, U. “A Contrail Cirrus Prediction Model.” Geoscientific Model Development 5, no. 3 (May 3, 2012): 543–80. https://doi.org/10.5194/gmd-5-543-2012.
- Schumann, U., B. Mayer, K. Graf, and H. Mannstein. “A Parametric Radiative Forcing Model for Contrail Cirrus.” Journal of Applied Meteorology and Climatology 51, no. 7 (July 2012): 1391–1406. https://doi.org/10.1175/JAMC-D-11-0242.1.
- Teoh, Roger, Ulrich Schumann, Arnab Majumdar, and Marc E. J. Stettler. “Mitigating the Climate Forcing of Aircraft Contrails by Small-Scale Diversions and Technology Adoption.” Environmental Science & Technology 54, no. 5 (March 3, 2020): 2941–50. https://doi.org/10.1021/acs.est.9b05608.
- Teoh, Roger, Ulrich Schumann, Edward Gryspeerdt, Marc Shapiro, Jarlath Molloy, George Koudis, Christiane Voigt, and Marc Stettler. “Aviation Contrail Climate Effects in the North Atlantic from 2016&ndash;2021.” Atmospheric Chemistry and Physics Discussions, March 30, 2022, 1–27. https://doi.org/10.5194/acp-2022-169.

In [4]:
import pandas as pd

from pycontrails import Flight
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.cocip import Cocip
from pycontrails.models.humidity_scaling import ConstantHumidityScaling
from pycontrails.physics import units

## Load Flight

Load flight trajectory from dataset prepared by Roger Teoh in https://doi.org/10.5194/acp-2022-169

In [5]:
# load flight waypoints
df_flight = pd.read_csv("/content/sample_data/flight-cocip.csv")
df_flight.head()

,Longitude (degrees),Latitude (degrees),Altitude (feet),UTC time,True airspeed (m s-1),Mach Number,Aircraft mass (kg),Fuel mass flow rate (kg s-1),Overall propulsion efficiency,nvPM number emissions index (kg-1),ICAO Aircraft Type,Wingspan (m)
0,-10.070,55.185,36000,1546651185,230.858,0.791,236479.000,1.654,0.4,1500000000000000,A359,64.75
1,-10.273,55.222,36000,1546651245,230.682,0.790,236379.755,1.657,0.4,1500000000000000,A359,64.75
2,-10.476,55.258,36000,1546651305,230.563,0.789,236280.355,1.659,0.4,1500000000000000,A359,64.75
3,-10.680,55.295,36000,1546651365,230.501,0.789,236180.791,1.661,0.4,1500000000000000,A359,64.75
4,-10.883,55.331,36000,1546651425,230.476,0.789,236081.128,1.662,0.4,1500000000000000,A359,64.75


In [6]:
# constant properties along the length of the flight
attrs = {
    "flight_id": "fid",
    "aircraft_type": df_flight["ICAO Aircraft Type"].values[0],
    "wingspan": df_flight["Wingspan (m)"].values[0],
}

Process the flight into a format expected by `pycontrails`. See [pycontrails.Flight](https://py.contrails.org/api/pycontrails.Flight.html#pycontrails.Flight) for interface details.

In [7]:
# convert UTC timestamp to np.datetime64
df_flight["time"] = pd.to_datetime(df_flight["UTC time"], origin="unix", unit="s")

# set altitude in m
df_flight["altitude"] = units.ft_to_m(df_flight["Altitude (feet)"])

# rename a few columns for compatibility with `Flight` requirements
df_flight = df_flight.rename(
    columns={
        "Longitude (degrees)": "longitude",
        "Latitude (degrees)": "latitude",
        "True airspeed (m s-1)": "true_airspeed",
        "Mach Number": "mach_number",
        "Aircraft mass (kg)": "aircraft_mass",
        "Fuel mass flow rate (kg s-1)": "fuel_flow",
        "Overall propulsion efficiency": "engine_efficiency",
        "nvPM number emissions index (kg-1)": "nvpm_ei_n",
    }
)

# clean up a few columns before building Flight class
df_flight = df_flight.drop(
    columns=["ICAO Aircraft Type", "Wingspan (m)", "UTC time", "Altitude (feet)"]
)

fl = Flight(data=df_flight, attrs=attrs)
fl

Flight [10 keys x 162 length, 3 attributes]
	Keys: longitude, latitude, true_airspeed, mach_number, aircraft_mass, ..., time
	Attributes:
	time                [2019-01-05 01:19:45, 2019-01-05 04:00:21]
	longitude           [-50.0, -10.07]
	latitude            [55.185, 61.089]
	altitude            [10972.8, 10972.8]
	flight_id           fid
	aircraft_type       A359
	wingspan            64.75

## Load meteorology from ECMWF

In [28]:
# get met domain from Flight
time = (
    pd.to_datetime(fl["time"][0]).floor("H"),
    pd.to_datetime(fl["time"][-1]).ceil("H") + pd.Timedelta("10H"),
)

# select pressure levels
pressure_levels = [
    400,
    350,
    300,
    250,
    225,
    200,
    175,
    150,
]

<ipython-input-28-703c643b8a43>:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.to_datetime(fl["time"][0]).floor("H"),
<ipython-input-28-703c643b8a43>:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.to_datetime(fl["time"][-1]).ceil("H") + pd.Timedelta("10H"),
<ipython-input-28-703c643b8a43>:4: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  pd.to_datetime(fl["time"][-1]).ceil("H") + pd.Timedelta("10H"),


In [49]:
!pip install netcdf4


In [50]:
# downloads met data from CDS
era5pl = ERA5(time=time, variables=Cocip.met_variables, pressure_levels=pressure_levels)
era5sl = ERA5(
    time=time,
    variables=Cocip.rad_variables,
)
met = era5pl.open_metdataset()
# rad = era5sl.open_metdataset(engine = 'h5netcdf')

print(time)
print(Cocip.met_variables)
print(pressure_levels)
# time = ("2022-03-01 00:00:00", "2022-03-01 03:00:00")
# variables = ["t", "q", "u", "v", "w", "ciwc", "z", "cc"]
# pressure_levels = [300, 250, 200]

2025-02-11 20:48:27,668 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-11 20:48:27,682 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-11 20:48:28,167 INFO Request ID is ce01b0e6-86df-4fe2-bedf-c4097e1bc8c0
INFO:datapi.legacy_api_client:Request ID is ce01b0e6-86df-4fe2-bedf-c4097e1bc8c0
2025-02-11 20:48:28,368 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-02-11 20:48:33,758 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been upd

27d2dcc652e62fb748b008cb05644ec8.nc:   0%|          | 0.00/58.3M [00:00<?, ?B/s]

ValueError: unrecognized engine 'netcdf4' must be one of your download engines: ['h5netcdf', 'scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [53]:
!pip install cdsapi

In [41]:
with open('/root/.cdsapirc') as f:
  print(f.read())

url:https://cds.climate.copernicus.eu/api
key:0956d699f29b6154c34148315839248c


In [ ]:
Timestamp('2019-01-05 01:00:00'), Timestamp('2019-01-05 15:00:00'))
(MetVariable(short_name='t', standard_name='air_temperature', long_name='Air Temperature', level_type='isobaricInhPa', ecmwf_id=130, grib1_id=11, grib2_id=(0, 0, 0), units='K', amip='ta', description='Air temperature is the bulk temperature of the air, not the surface (skin) temperature.'), MetVariable(short_name='q', standard_name='specific_humidity', long_name='Specific Humidity', level_type='isobaricInhPa', ecmwf_id=133, grib1_id=51, grib2_id=(0, 1, 0), units='kg kg**-1', amip='hus', description='Specific means per unit mass. Specific humidity is the mass fraction of water vapor in (moist) air.'), MetVariable(short_name='u', standard_name='eastward_wind', long_name='Eastward Wind', level_type='isobaricInhPa', ecmwf_id=131, grib1_id=33, grib2_id=(0, 2, 2), units='m s**-1', amip='ua', description='"Eastward" indicates a vector component which is positive when directed eastward (negative westward). Wind is defined as a two-dimensional (horizontal) air velocity vector, with no vertical component.'), MetVariable(short_name='v', standard_name='northward_wind', long_name='Northward Wind', level_type='isobaricInhPa', ecmwf_id=132, grib1_id=34, grib2_id=(0, 2, 3), units='m s**-1', amip='va', description='"Northward" indicates a vector component which is positive when directed northward (negative southward). Wind is defined as a two-dimensional (horizontal) air velocity vector, with no vertical component.'), MetVariable(short_name='w', standard_name='lagrangian_tendency_of_air_pressure', long_name='Vertical Velocity (omega)', level_type='isobaricInhPa', ecmwf_id=135, grib1_id=39, grib2_id=(0, 2, 8), units='Pa s**-1', amip='wap', description='The Lagrangian tendency of air pressure, often called "omega", plays the role of the upward component of air velocity when air pressure is being used as the vertical coordinate. If the vertical air velocity is upwards, it is negative when expressed as a tendency of air pressure; downwards is positive. Air pressure is the force per unit area which would be exerted when the moving gas molecules of which the air is composed strike a theoretical surface of any orientation.'), (MetVariable(short_name='cli', standard_name='mass_fraction_of_cloud_ice_in_air', long_name='Mass fraction of cloud ice in air', level_type='isobaricInhPa', ecmwf_id=None, grib1_id=None, grib2_id=None, units='kg kg**-1', amip='cli', description='The mass fraction of cloud ice in moist air.'), MetVariable(short_name='ciwc', standard_name='specific_cloud_ice_water_content', long_name='Specific cloud ice water content', level_type='isobaricInhPa', ecmwf_id=247, grib1_id=None, grib2_id=(0, 1, 84), units='kg kg**-1', amip=None, description="This parameter is the mass of cloud ice particles per kilogram of the total mass of moist air. The 'total mass of moist air' is the sum of the dry air, water vapour, cloud liquid, cloud ice, rain and falling snow. This parameter represents the average value for a grid box."), MetVariable(short_name='icmr', standard_name='ice_water_mixing_ratio', long_name='Cloud ice water mixing ratio', level_type='isobaricInhPa', ecmwf_id=260019, grib1_id=None, grib2_id=(0, 1, 23), units='kg kg**-1', amip=None, description='This parameter is the mass of cloud ice particles per kilogram of the total mass of dry air. ')))
[400, 350, 300, 250, 225, 200, 175, 150]

In [43]:
import cdsapi
import xarray as xr

c = cdsapi.Client()
        # 'variable': ['air_temperature', 'specific_humidity', 'eastward_wind', 'northward_wind', 'lagrangian_tendency_of_air_pressure', 'mass_fraction_of_cloud_ice_in_air', 'specific_cloud_ice_water_content', 'ice_water_mixing_ratio'],

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': ['specific_humidity' ],
        'pressure_level': ['400', '350', '300', '250', '225', '200', '175', '150'],
        'year': '2019',
        'month': '01',
        'day': '05',
        'time': ['01:00', '15:00'],
        'format': 'netcdf'
    },
    'download.nc')

ds = xr.open_dataset('download.nc')

2025-02-11 20:34:19,728 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-11 20:34:19,733 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-11 20:34:20,195 INFO Request ID is 5943b55b-4d0e-4843-b123-e0d41148d7c5
INFO:datapi.legacy_api_client:Request ID is 5943b55b-4d0e-4843-b123-e0d41148d7c5
2025-02-11 20:34:20,381 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-02-11 20:34:29,287 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been 

bae4ca9012a0a84a3dc665dc14b7ea32.nc:   0%|          | 0.00/27.4M [00:00<?, ?B/s]

In [23]:
!ping cds.climate.copernicus.eu

/bin/bash: line 1: ping: command not found


In [38]:
!pip install netCDF4

In [40]:
from pycontrails.datalib.ecmwf import ERA5
import netCDF4
import numpy as np
import pandas as pd
import xarray as xr
year= ["2024"]
month = ["02"]
day= ["10"]
# time= ["06:00", "07:00", "08:00"]
# time = ("2022-03-01 00:00:00", "2022-03-01 23:00:00")
time = "2020-06-01 12:00:00"



pressure_levels = [350, 300, 250, 225, 200, 175, 150]
met_variables = ["air_temperature", "relative_humidity"]
rad_variables = ["tsr", "ttr"]

era5 = ERA5( time= time, variables=met_variables, pressure_levels=pressure_levels)
print(era5)
ds = xr.open_mfdataset(
    "/specific_humidity.nc",
    engine='h5netcdf'
)
print(ds)

ERA5
	Timesteps: ['2020-06-01 12']
	Variables: ['t', 'r']
	Pressure levels: [150, 175, 200, 225, 250, 300, 350]
	Grid: 0.25
	Dataset: reanalysis-era5-pressure-levels
	Product type: reanalysis
<xarray.Dataset> Size: 66MB
Dimensions:         (valid_time: 2, pressure_level: 8, latitude: 721,
                     longitude: 1440)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 16B 2019-01-05T01:00:00 2019-...
  * pressure_level  (pressure_level) float64 64B 400.0 350.0 ... 175.0 150.0
  * latitude        (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude       (longitude) float64 12kB 0.0 0.25 0.5 ... 359.2 359.5 359.8
    expver          (valid_time) <U4 32B dask.array<chunksize=(2,), meta=np.ndarray>
Data variables:
    q               (valid_time, pressure_level, latitude, longitude) float32 66MB dask.array<chunksize=(1, 4, 361, 720), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription

In [16]:
type(era5pl)

pycontrails.datalib.ecmwf.era5.ERA5

## Set up model

In [ ]:
params = {
    "process_emissions": False,
    "verbose_outputs": True,
    "humidity_scaling": ConstantHumidityScaling(rhi_adj=0.98),
}
cocip = Cocip(met=met, rad=rad, params=params)

## Run model

In [ ]:
fl_out = cocip.eval(source=fl)

## Review output

The output flight has the original flight data with many new variables added from the evaluation.

In [ ]:
fl_out

In [ ]:
fl_out.dataframe.columns

The `cocip` variable describes where persistent contrails form. It can take on values:

- 1: Persistent contrails form
- 0: No persistent contrails form

In [ ]:
fl_out["cocip"]

The model class contains information about the contrail created:

- `cocip.source` the original input flight
- `cocip.contrail` will be defined as a `pandas` DataFrame if a contrail is created.
- `cocip.contrail_dataset` is the same data but formatted as an `xarray` Dataset.

In [ ]:
cocip.contrail

We can visualize the contrail on top of the original flight trajectory using pandas plotting capabilities

In [ ]:
ax = cocip.source.dataframe.plot(
    "longitude", "latitude", color="k", label=fl.attrs["flight_id"], figsize=(12, 8)
)
cocip.contrail.plot.scatter("longitude", "latitude", c="rf_lw", cmap="Reds", ax=ax);

In [ ]:
ax = cocip.source.dataframe.plot(
    "longitude", "latitude", color="k", label=fl.attrs["flight_id"], figsize=(12, 8)
)
cocip.contrail.plot.scatter(
    "longitude", "latitude", c="ef", cmap="coolwarm", vmin=-1e12, vmax=1e12, ax=ax
);